In [1]:
%config Completer.use_jedi = False
import sys
import warnings
warnings.filterwarnings('ignore')

import os
os.environ["CUDA_VISIBLE_DEVICES"]="2"

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

2023-01-28 13:37:27.687103: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-28 13:37:27.906958: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-01-28 13:37:28.556759: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /end/home/rh2310/lib:/end/soft/Linux_2.6_64/lib::/end/home/rh2310/.conda/envs/tensorflo

In [2]:
MEMORY = 8*1024

gpus = tf.config.list_physical_devices('GPU')
try:
    tf.config.set_logical_device_configuration(gpus[0],
                                              [tf.config.LogicalDeviceConfiguration(memory_limit=MEMORY)])
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
except RuntimeError as e:        
    # Memory growth must be set before GPUs have been initialized
    print(e) 

1 Physical GPUs, 1 Logical GPUs


2023-01-28 13:37:37.507439: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-28 13:37:38.229449: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 8192 MB memory:  -> device: 0, name: NVIDIA RTX A6000, pci bus id: 0000:e3:00.0, compute capability: 8.6


In [13]:
sys.path.append("/end/home/rh2310/morpho_repo/turing_codebase")
from turing.utils import *
from turing.tf_utils import *
import turing.pinns as tu
from turing.loss_functions import *

In [14]:
from local_utils import *

In [5]:
optimizer=keras.optimizers.Adam(learning_rate=5e-4)
layers = [2, 128, 128, 128, 128, 128, 2]
obs_loss = Observation_Loss(layers, loss_grad_type=  Loss_Grad_Type.PINN)
pinn = tu.NN(layers, lb=0, ub=10, dtype=tf.float32)
norm_l2 = L2()

inputs = np.concatenate([np.arange(0, 10)[:, np.newaxis], 
                         np.arange(0, 10)[:, np.newaxis]], axis=1)

def f1(X, alpha):
    return np.exp(alpha*np.abs(X[:,0]-X[:,1]))

outputs = np.concatenate([f1(inputs, 1)[:, np.newaxis],
                          f1(inputs, 3)[:, np.newaxis]], axis=1)
obs_X = np.concatenate([inputs, outputs], axis=1)

ds = TINN_Dataset(tf.float32, obs_X)
ds2 = TINN_Dataset(tf.float32, obs_X, obs_X)

@tf.function
def train_step(elements):
    with tf.GradientTape(persistent=True) as tape:        
        items = obs_loss.residual(pinn, elements[0])
        l2_items = norm_l2.reduce_norm(items)
        l2 = tf.reduce_sum(l2_items)
        trainables = pinn.trainable_variables
        
    grads = tape.gradient(l2, trainables)
    optimizer.apply_gradients(zip(grads, trainables))
    return l2

@tf.function
def train_step_double_loss(elements):
    with tf.GradientTape(persistent=True) as tape:        
        items1 = obs_loss.residual(pinn, elements[0])
        l2_items1 = norm_l2.reduce_norm(items1)
        l2_1 = tf.reduce_sum(l2_items1)
        items2 = obs_loss.residual(pinn, elements[1])
        l2_items2 = norm_l2.reduce_norm(items2)
        l2_2 = tf.reduce_sum(l2_items2)
        l2 = l2_1 + l2_2
        trainables = pinn.trainable_variables
        
    grads = tape.gradient(l2, trainables)
    optimizer.apply_gradients(zip(grads, trainables))
    return l2


def epoch_callback(epoch, loss_samples):
    if epoch%10 == 0:
        print("Callback called")

losses = tu.train(epochs=100,
                  batch_size=2,
                  dataset = ds ,
                  train_step = train_step,
                  print_interval=20,
                  loss_sample_interval = 20,
                  stop_threshold=1e-12,
                  epoch_callback=epoch_callback)

losses2 = tu.train(epochs=100,
                  batch_size=2,
                  dataset = ds2 ,
                  train_step = train_step_double_loss,
                  print_interval=20,
                  loss_sample_interval = 20,
                  stop_threshold=1e-12,
                  epoch_callback=epoch_callback)


Start of epoch 0
Callback called
Loss value: 3.2259960174560547, at epoch 0
Time taken: 2.41s
Callback called

Start of epoch 20
Callback called
Loss value: 2.1694495444535278e-05, at epoch 20
Time taken: 0.73s
Callback called

Start of epoch 40
Callback called
Loss value: 9.693593892734498e-05, at epoch 40
Time taken: 0.73s
Callback called

Start of epoch 60
Callback called
Loss value: 9.845380554907024e-05, at epoch 60
Time taken: 0.75s
Callback called

Start of epoch 80
Callback called
Loss value: 0.00010113465395988896, at epoch 80
Time taken: 0.71s
Callback called
